
## Spectral graph theory

A graph $G$ is a pair of related sets $V$ and $E$ where$|V|=n$. The elements of $E$ are 
un-ordered pairs of elements of $V$ with no repeats. A particular vertex is often called
$u$ or $v$ and each vertex has degree $d_u$. We disallow loops and multiple edges at the
outset, noting that multiple edges can be interpreted as edge weights in future developments. 



A graph may have multiple connected components.



There is a lot of additional preliminary terminology to set up.



* $vol\;G\;=\;\sum_{v}{d_v}$



- A graph can be represented as either a Laplacian or a normalized Laplacian $n\;\times\;n$ matrix
- For an un-directed graph this will be real symmetric and will have real non-negative eigenvalues
- These $n$ eigenvalues are sorted in increasing order $\{\lambda_0,\;\lambda_1,\;\lambda_2\;\dots\;\lambda_{n-1}\}$
    - ...using Chung's subscript notation beginning at 0
- The number of 0 eigenvalues corresponds to the number of connected components.


Define a cut as a (minimal) collection of cut edges to break the graph into disconnected sub-graphs.
Cuts indicate relatively low connectivity.

Isoperimetric ratio $\theta(S)$ I refer to as IPR. 

Important in getting to Cheeger's inequality: Rayleigh quotients of eigenvalues orthogonal to
the $1$ vector corresponding to $\lambda_0=0$. The RQ is used in an upper bound on IPR. The 
second eigenvalue $\lambda_1$ is used in like fashion to give an upper bound of the IPR of G. 
That is we want to find vertex subsets S with low isoperimetry.

Something like $\theta(S_t)\le \sqrt{2d\lambda_1}$.

It would be helpful to work some examples. 

Cheeger's inequality can be generalized for k-clustering. 
 

In [1]:
import sys, time                              # can be used to halt programs that run long
import numpy as np                            # linear algebra
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import random as r                            # method random(): uniform distribution on (0, 1)
from numpy import zeros
import warnings; warnings.simplefilter('ignore')

In [2]:
def eigensketch(G, threshold = 9):
    import numpy.linalg

    n = len(G.nodes())
    L = nx.normalized_laplacian_matrix(G)
    e, v = numpy.linalg.eig(L.A)
    e.sort()

    e = [e[i] if e[i] > 1e-14 else 0 for i in range(len(e))]
    msg = "Eigenvalues: "
    nTerms = min(n, threshold)
    msg += '%.2f' % e[0]
    for i in range(1, nTerms): msg += ', ' + '%.2f' % e[i]
    if   n == threshold + 1: msg += ', ' + '%.2f' % e[-1]
    elif n == threshold + 2: msg += ', ' + '%.2f' % e[-2] + ', ' + '%.2f' % e[-1]
    elif n  > threshold + 2: msg += ', ..., ' + '%.2f' % e[-2] + ', ' + '%.2f' % e[-1]

    print(msg)
    
def hamiltonian(G):
    F = [(G,[list(G.nodes())[0]])]   # A one-element list -> a 2-ple [(G, label of node 0 of G)]
    n = G.number_of_nodes()
    while F:
        graph,path = F.pop()
        confs = []
        neighbors = (node for node in graph.neighbors(path[-1]) if node != path[-1]) #exclude self loops
        for neighbor in neighbors:
            conf_p = path[:]
            conf_p.append(neighbor)
            conf_g = nx.Graph(graph)
            conf_g.remove_node(path[-1])
            confs.append((conf_g,conf_p))
        for g,p in confs:
            if len(p) == n: return p
            else:           F.append((g,p))
    return None

In [8]:
G = nx.random_geometric_graph(20, 0.60)
F = hamiltonian(G)
print(F)

[0, 19, 18, 17, 14, 15, 16, 10, 13, 12, 11, 9, 7, 6, 8, 5, 4, 3, 2, 1]


In [9]:
eigensketch(G, 18)

Eigenvalues: 0.00, 0.66, 0.79, 0.95, 0.97, 1.02, 1.03, 1.04, 1.05, 1.06, 1.06, 1.07, 1.09, 1.11, 1.12, 1.14, 1.17, 1.20, 1.22, 1.26


In [10]:
nx.laplacian_spectrum(G)/10

array([-1.40191851e-16,  7.45345291e-01,  8.62763048e-01,  9.98400246e-01,
        1.08022308e+00,  1.21991662e+00,  1.23789054e+00,  1.29094787e+00,
        1.30978438e+00,  1.39224711e+00,  1.46048072e+00,  1.59501395e+00,
        1.64842409e+00,  1.71816614e+00,  1.79835075e+00,  1.82965992e+00,
        1.90000000e+00,  1.91238623e+00,  2.00000000e+00,  2.00000000e+00])

```
F = hamiltonian(G)    # run with 20 vertices and p = 0.4; single component graph
F                     # will give an H path (not cycle)

[0, 3, 4, 16, 19, 17, 14, 13, 11, 12, 18, 9, 8, 5, 7, 1, 10, 6, 15, 2]
```

Let's decipher the brute force code above. $n$ is the number of nodes in $G$. Here is a paraphrase of a repl: 

```
F = [(G,[list(G.nodes())[0]])]
print(type(F[0]), len(F[0]), type(F[0][0]), type(F[0][1]), len(F[0][1]), type(F[0][1][0]), F[0][1][0])

'tuple', 2, graph G, 'list', 1, 'int', 0
```

From this we see that F is a list of one element; which is a 2-ple: The passed graph G and (second) a list of length 1, with list[0] = 0. 
This zero is the index of the first element of G's node list, i.e. the start of a path. The second element of the tuple (i.e. the list) 
will prove to be a path of vertices. 


The `while` loop runs until `F` is empty.


`graph, path = F.pop()` breaks out the `F` tuple into `graph` and `path`; leaving `F` an empty list after the pop(). Remember `path` will grow.


`confs` is an empty list that grows as we go. It accumulates (Graph, path-list) tuples.


`neighbors` is a list of nodes that are neighbors to the last node on the working `path`... excluding that node itself. This implies that if 
G has self-loops then `.neighbors(node)` would include 'myself'.  

Incidentally `G.neighbors(node)` returns an enumerable; I found it expedient to use it like `range()`: 

```
for i in G.neighbors(4): print(i)

9 18 11 13 6 8 17 19 1 3 12 5 14 7 16
```

As the while progresses brute force additions are made to generate new paths and graphs that have the newest path vertext removed. 
I'll leave it at that for now but it is an interesting bit of code. 

In [13]:
import inspect
print(inspect.getdoc(np.linalg.eig))

Compute the eigenvalues and right eigenvectors of a square array.

Parameters
----------
a : (..., M, M) array
    Matrices for which the eigenvalues and right eigenvectors will
    be computed

Returns
-------
A namedtuple with the following attributes:

eigenvalues : (..., M) array
    The eigenvalues, each repeated according to its multiplicity.
    The eigenvalues are not necessarily ordered. The resulting
    array will be of complex type, unless the imaginary part is
    zero in which case it will be cast to a real type. When `a`
    is real the resulting eigenvalues will be real (0 imaginary
    part) or occur in conjugate pairs

eigenvectors : (..., M, M) array
    The normalized (unit "length") eigenvectors, such that the
    column ``eigenvectors[:,i]`` is the eigenvector corresponding to the
    eigenvalue ``eigenvalues[i]``.

Raises
------
LinAlgError
    If the eigenvalue computation does not converge.

See Also
--------
eigvals : eigenvalues of a non-symmetric array.
eigh

In [14]:
import inspect
print(inspect.getdoc(nx.random_graphs.erdos_renyi_graph))

Returns a $G_{n,p}$ random graph, also known as an Erdős-Rényi graph
or a binomial graph.

The $G_{n,p}$ model chooses each of the possible edges with probability $p$.

Parameters
----------
n : int
    The number of nodes.
p : float
    Probability for edge creation.
seed : integer, random_state, or None (default)
    Indicator of random number generation state.
    See :ref:`Randomness<randomness>`.
directed : bool, optional (default=False)
    If True, this function returns a directed graph.

See Also
--------
fast_gnp_random_graph

Notes
-----
This algorithm [2]_ runs in $O(n^2)$ time.  For sparse graphs (that is, for
small values of $p$), :func:`fast_gnp_random_graph` is a faster algorithm.

:func:`binomial_graph` and :func:`erdos_renyi_graph` are
aliases for :func:`gnp_random_graph`.

>>> nx.binomial_graph is nx.gnp_random_graph
True
>>> nx.erdos_renyi_graph is nx.gnp_random_graph
True

References
----------
.. [1] P. Erdős and A. Rényi, On Random Graphs, Publ. Math. 6, 290 (1959

In [30]:
print(inspect.getdoc(nx.draw))

Draw the graph G with Matplotlib.

Draw the graph as a simple representation with no node
labels or edge labels and using the full Matplotlib figure area
and no axis labels by default.  See draw_networkx() for more
full-featured drawing that allows title, axis labels etc.

Parameters
----------
G : graph
    A networkx graph

pos : dictionary, optional
    A dictionary with nodes as keys and positions as values.
    If not specified a spring layout positioning will be computed.
    See :py:mod:`networkx.drawing.layout` for functions that
    compute node positions.

ax : Matplotlib Axes object, optional
    Draw the graph in specified Matplotlib axes.

kwds : optional keywords
    See networkx.draw_networkx() for a description of optional keywords.

Examples
--------
>>> G = nx.dodecahedral_graph()
>>> nx.draw(G)
>>> nx.draw(G, pos=nx.spring_layout(G))  # use spring layout

See Also
--------
draw_networkx
draw_networkx_nodes
draw_networkx_edges
draw_networkx_labels
draw_networkx_edge_l